# Chapter 6. 행렬과 벡터 계산

## 6.1 문제 소개

이 장에서는 유체 확산 예제를 반복해서 사용하며 행렬과 벡터 연산을 살펴본다. <br>
<br>
확산 방정식은 다음과 같다. <br>
$$
\frac{\partial}{\partial t} u(x,t) = D \cdot \frac{\partial^2}{\partial x^2} u(x,t)
$$ <br>
여기서 $ u $는 확산하는 유체의 양을 나타내는 벡터이고, $ D $는 확산 계수이다. 예제가 복잡해지지 않도록 코드에서는 $ D = 1 $로 놓는다. <br>
<br>
시간과 공간상에서 연속적인 확산 방정식을 이용해서 불연속적인 공간과 시간으로 근삿값을 구하려한다. 이때 오일러 방법(Euler's Method)을 이용한다. 오일러 방법은 다음과 같다. <br>
$$
\frac{\partial}{\partial t} u(x,t) \approx \frac{u(x,t+dt) - u(x,t)}{dt}
$$ <br>
여기서, $ u(x, t+dt) $는 유한 차분법을 이용해서 다음과 같이 계산할 수 있다. <br>
$$
u(x, t+dt) = u(x,t) + dt \star D \star \frac{u(x+dx,t) + u(x-dx,t) - 2 u(x,t)}{dx^2}
$$

**예제 6-1** 단순(1차원) 확산 의사 코드

In [ ]:
# 초기 조건을 생성한다.
u = vector of length N
for i in range(N):
    u = 0 if there is water, 1 if there is dye
    
# 초기 조건을 변경한다.
D = 1
t = 0
dt = 0.0001
while True:
    print(f"Current time is: {t}")
    unew = vector of size N
    
    # 행렬의 각 셀을 갱신한다.
    for i in range(N):
        unew[i] = u[i] + D * dt * (u[(i+1) % N] + u[(i-1) % N] - * u[i])
    # 갱신된 해법을 u로 옮긴다.
    u = unew
    
    visualize(u)

2차원 확산 방정식은 다음과 같다. <br>
<br>
$$
\frac{\partial}{\partial t} u(x,y,t) = D \cdot \left( \frac{\partial^2}{\partial x^2} u(x,y,t) + \frac{\partial^2}{\partial y^2} u(x,y,t) \right)
$$

**예제 6-2** 2차원 확산을 계산하는 알고리즘

In [ ]:
for i in range(N):
    for j in range(M):
        unew[i][j] = u[i][j] + dt * (
            (u[(i + 1) % N][j] + u[(i - 1) % N][j] - 2 * u[i][j]) + # d^2 u / dx^2
            (u[i][(j + 1) % M] + u[i][(j - 1) % M] - 2 * u[i][j])   # d^2 u / dy^2
        )

## 6.2 파이썬의 리트스만으로 충분할까?

**예제 6-3** 순수 파이썬으로 작성한 2차원 확산

In [ ]:
grid_shape = (640, 640)

# 행렬을 받아 변화된 상태를 반환하는 함수.
def evolve(grid, dt, D=1.0):
    xmax, ymax = grid_shape
    new_grid = [[0.0 for y in range(grid_shape[1])] for x in range(grid_shape[0])]
    for i in range(xmax):
        for j in range(ymax):
            grid_xx = (
                grid[(i + 1) % xmax][j] + grid[(i - 1) % xmax][j] - 2.0 * grid[i][j]
            )
            grid_yy = (
                grid[i][(j + 1) % ymax] + grid[i][(j - 1) % ymax] - 2.0 * grid[i][j]
            )
            new_grid[i][j] = grid[i][j] + D * (grid_xx + grid_yy) * dt
    return new_grid

**예제 6-4** 순수 파이썬으로 작성한 2차원 확산 방정식의 초기화

In [ ]:
def run_experiment(num_iterations):
    # 초기 조건을 설정한다.
    grid = [[0.0 for y in range(grid_shape[1])] for x in range(grid_shape[0])]

    # 시뮬레이션 영역의 중간에 물감이 한 방울
    # 떨어진 상태를 시뮬레이션하기 위한 초기 조건들
    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)
    for i in range(block_low, block_high):
        for j in range(block_low, block_high):
            grid[i][j] = 0.005

    # 초기 조건을 변경한다.
    start = time.time()
    for i in range(num_iterations):
        grid = evolve(grid, 0.1)
    return time.time() - start

**예제 6-5** 순수 파이썬 2차원 확산 방정식 프로파일링

<span style="font-family: monospace">
~# kernprof -lv diffusion_python.py
</span>

In [ ]:
Wrote profile results to diffusion_python.py.lprof
Timer unit: 1e-06 s

Total time: 786.236 s
File: diffusion_python.py
Function: evolve at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    13                                           @profile
    14                                           def evolve(grid, dt, D=1.0):
    15       500        935.0      1.9      0.0      xmax, ymax = grid_shape
    16       500   19099559.0  38199.1      2.4      new_grid = [[0.0 for x in range(grid_shape[1])] for x in range(grid_shape[0])]
    17    320500     172347.0      0.5      0.0      for i in range(xmax):
    18 205120000   97443356.0      0.5     12.4          for j in range(ymax):
    19 204800000   97762149.0      0.5     12.4              grid_xx = (
    20 204800000  169351245.0      0.8     21.5                  grid[(i + 1) % xmax][j] + grid[(i - 1) % xmax][j] - 2.0 * grid[i][j]
    21                                                       )
    22 204800000   95734000.0      0.5     12.2              grid_yy = (
    23 204800000  167381414.0      0.8     21.3                  grid[i][(j + 1) % ymax] + grid[i][(j - 1) % ymax] - 2.0 * grid[i][j]
    24                                                       )
    25 204800000  139290578.0      0.7     17.7              new_grid[i][j] = grid[i][j] + D * (grid_xx + grid_yy) * dt
    26       500        335.0      0.7      0.0      return new_grid

프로파일링 결과에 따르면 도함수를 계산하고 행렬의 값을 갱신하는데 대부분의 시간이 소비됐다. <br>
<br>
Line 15는 실행될 때마다 시간을 많이 잡아먹는데, 이는 grid_shape를 지역 네임스페이스에서 가져와야 하기 때문이다. <br>
<br>
Line 18은 320,500번 실행되었다. 처리하는 격자의 xmax = 640이고 함수를 500번 실행했기 때문이다. 횟수는 (640 + 1) * 500으로 계산할 수 있는데, 루프 종료 조건을 만나려고 추가로 한 번 더 실행되기 때문이다. <br>
<br>
Line 26은 500번 실행되었다. 즉 함수를 500번 실행해 프로파일 했다는 뜻이다. <br>
<br>
Line 16에서 Per Hit와 % Time의 차이가 큰데, 이는 이 줄이 실행하는 데 시간이 꽤 걸리는 반면, 다른 줄에 비해 훨씬 덜 호출되기 때문이다. <br>
<br>
evolve에 어떤 값이 들어오든 new_grid 리스트는 항상 모양과 크기, 저장값이 같으므로, 리스트를 한 번만 저장하고 재활용하는 방법으로 최적화할 수 있다. (반복되는 코드를 루프 밖으로 빼내는 작업)

**예제 6-6** 메모리 할당을 줄인 순수 파이썬 2차원 확산 방정식 코드

In [ ]:
@profile
def evolve(grid, dt, out, D=1.0):
    xmax, ymax = grid_shape
    for i in range(xmax):
        for j in range(ymax):
            grid_xx = (
                grid[(i + 1) % xmax][j] + grid[(i - 1) % xmax][j] - 2.0 * grid[i][j]
            )
            grid_yy = (
                grid[i][(j + 1) % ymax] + grid[i][(j - 1) % ymax] - 2.0 * grid[i][j]
            )
            out[i][j] = grid[i][j] + D * (grid_xx + grid_yy) * dt


def run_experiment(num_iterations):
    # setting up initial conditions
    scratch = [[0.0 for x in range(grid_shape[1])] for x in range(grid_shape[0])]
    grid = [[0.0 for x in range(grid_shape[1])] for x in range(grid_shape[0])]

    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)
    for i in range(block_low, block_high):
        for j in range(block_low, block_high):
            grid[i][j] = 0.005

    start = time.time()
    for i in range(num_iterations):
        evolve(grid, 0.1, scratch)
        grid, scratch = scratch, grid
    return time.time() - start

**예제 6-7** 메모리 할당을 줄인 파이썬 확산 방정식 코드의 프로파일링 결과

<span style="font-family: monospace">
~# kernprof -lv diffusion_python_memory.py
</span>

In [ ]:
Wrote profile results to diffusion_python_memory.py.lprof
Timer unit: 1e-06 s

Total time: 528.467 s
File: diffusion_python_memory.py
Function: evolve at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    13                                           @profile
    14                                           def evolve(grid, dt, out, D=1.0):
    15       500        427.0      0.9      0.0      xmax, ymax = grid_shape
    16    320500     101954.0      0.3      0.0      for i in range(xmax):
    17 205120000   59498867.0      0.3     11.3          for j in range(ymax):
    18 204800000   58363954.0      0.3     11.0              grid_xx = (
    19 204800000  125491325.0      0.6     23.7                  grid[(i + 1) % xmax][j] + grid[(i - 1) % xmax][j] - 2.0 * grid[i][j]
    20                                                       )
    21 204800000   58815116.0      0.3     11.1              grid_yy = (
    22 204800000  128486419.0      0.6     24.3                  grid[i][(j + 1) % ymax] + grid[i][(j - 1) % ymax] - 2.0 * grid[i][j]
    23                                                       )
    24 204800000   97709289.0      0.5     18.5              out[i][j] = grid[i][j] + D * (grid_xx + grid_yy) * dt

## 6.3 메모리 단편화

파이썬의 리스트는 실제 데이터를 가리키는 포인터를 저장한다. 데이터 타입에 상관없이 리스트에 어떤 데이터도 저장할 수 있으므로 일반적으로 좋은 선택이지만, 벡터와 행렬 연산에서는 성능 저하의 원인이 된다.

**예제 6-8** 메모리 할당을 줄인 2차원 확산 파이썬 방정식 코드의 성능 측정 결과 (help!!)

<span style="font-family: monospace">
~# git clone --depth 1 https://git.kernel.org/pub/scm/linux/kernel/git/torvalds/linux.git <br>
~# cd linux/tools/perf <br>
~# make <br>
~# cp perf /usr/bin <br>
~# apt-get install build-essential git flex bison <br>
</span>

<span style="font-family: monospace">
~# perf stat -e cycles,instructions,cache-references,cache-misses,branches,branch-misses,task-clock,faults,minor-faults,cs,migrations python3 diffusion_python_memory.py
</span>

In [ ]:
 Performance counter stats for 'python3 diffusion_python_memory.py':

   <not supported>      cycles
   <not supported>      instructions
   <not supported>      cache-references
   <not supported>      cache-misses
   <not supported>      branches
   <not supported>      branch-misses
          94549.53 msec task-clock                #    1.000 CPUs utilized
              9650      faults                    #  102.063 /sec
              9650      minor-faults              #  102.063 /sec
                44      cs                        #    0.465 /sec
                 0      migrations                #    0.000 /sec

      94.562164800 seconds time elapsed

      94.552682000 seconds user
       0.000000000 seconds sys

- task-clock: 소비한 총 클럭 수를 나타낸다.
- instrunctions: 코드에서 얼마나 많은 CPU 명령어가 실행됐는지를 나타낸다.
- cycles: 명령어를 실행하는 데 CPU 사이클이 얼마나 걸렸는지를 나타낸다.
- cs: I/O 같은 커널 작업이 끝나기를 기다리거나, 다른 애플리케이션이 실행되는 동안 대기하거나, 다른 CPU 코어로 작업을 옮기려고 프로그램이 얼마나 오래 멈춰 있었는지를 알려준다.
- cache-references: 캐시의 데이터를 참조하면 증가하는 지표.
- cache-misses: 데이터가 캐시에 존재하지 않아서 RAM에서 읽어와야 하면 증가하는 지표.
- branch: 어떤 조건의 결과에 따라 코드의 이 부분이나 저 부분(코드 실행 시 발생하는 분기)을 실행할 수 있다. CPU는 분기가 어느 방향으로 일어날지 예측하고 적잘한 분기에 속한 명령을 미리 읽어온다.
- branch-misses: 분기 예측이 틀리면 branch-miss가 발생한다.
- fault: 메모리가 할당됐을 때, 커널은 메모리에 대한 참조를 프로그램에 알려주기만 한다. 하지만 나중에 그 메모리가 처음 사용되면 운영체제는 가벼운 페이지 폴트 인터럽트를 발생시켜 실행 중이던 프로그램을 잠시 멈추고 실제 필요한 메모리를 할당한다.

In [7]:
from array import array
import numpy

def norm_square_list(vector):
    norm = 0
    for v in vector:
        norm += v * v
    return norm

def norm_square_list_comprehension(vector):
    return sum([v * v for v in vector])

def norm_square_array(vector):
    norm = 0
    for v in vector:
        norm += v * v
    return norm

def norm_square_numpy(vector):
    return numpy.sum(vector * vector)

def norm_square_numpy_dot(vector):
    return numpy.dot(vector, vector)

In [9]:
vector = list(range(1_000_000))
%timeit norm_square_list(vector)

74.3 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
vector = list(range(1_000_000))
%timeit norm_square_list_comprehension(vector)

95 ms ± 2.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
vector_array = array('l', range(1_000_000))
%timeit norm_square_array(vector_array)

87.1 ms ± 8.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
vector_np = numpy.arange(1_000_000)
%timeit norm_square_numpy(vector_np)

1.77 ms ± 17.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
vector_np = numpy.arange(1_000_000)
%timeit norm_square_numpy_dot(vector_np)

640 µs ± 36.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 6.4 넘파이를 이용한 확산 방정식 해법

In [18]:
# np.roll(array, shitf, axis=None) 함수 소개
import numpy as np
np.roll([1,2,3,4], 1)

array([4, 1, 2, 3])

In [24]:
np.roll([[1,2,3], [4,5,6]], 1, axis=None)

array([[6, 1, 2],
       [3, 4, 5]])

In [19]:
np.roll([[1,2,3], [4,5,6]], 1, axis=0)

array([[4, 5, 6],
       [1, 2, 3]])

In [23]:
np.roll([[1,2,3], [4,5,6]], 1, axis=1)

array([[3, 1, 2],
       [6, 4, 5]])

**예제 6-9** numpy를 사용한 파이썬 확산 방정식 최초 버전

In [ ]:
from numpy import roll, zeros

grid_shape = (640, 640)

def laplacian(grid):
    return (
        roll(grid, +1, 0)
        + roll(grid, -1, 0)
        + roll(grid, +1, 1)
        + roll(grid, -1, 1)
        - 4 * grid
    )

def evolve(grid, dt, D=1):
    return grid + dt * D * laplacian(grid)

def run_experiment(num_iterations):
    grid = zeros(grid_shape)

    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)
    grid[block_low:block_high, block_low:block_high] = 0.005

    for i in range(num_iterations):
        grid = evolve(grid, 0.1)
    return grid

**예제 6-10** numpy를 사용한 2차원 확산 방정식의 성능 측정(격자 크기: 640x640, 500회 반복)

<span style="font-family: monospace">
~# perf stat -e cycles,instructions,cache-references,cache-misses,branches,branch-misses,task-clock,faults,minor-faults,cs,migrations python3 diffusion_numpy.py
</span>

In [ ]:
 Performance counter stats for 'python diffusion_numpy.py':

     8,432,416,866      cycles                    #    2.886 GHz                      
     7,114,758,602      instructions              #    0.84  insn per cycle           
     1,040,831,469      cache-references          #  356.176 M/sec                    
       216,490,683      cache-misses              #   20.800 % of all cache refs      
     1,252,928,847      branches                  #  428.756 M/sec                    
         8,174,531      branch-misses             #    0.65% of all branches          
       2922.239426      task-clock (msec)         #    1.285 CPUs utilized          
           403,282      faults                    #    0.138 M/sec                  
           403,282      minor-faults              #    0.138 M/sec                  
                96      cs                        #    0.033 K/sec                  
                 5      migrations                #    0.002 K/sec                  

       2.274377105 seconds time elapsed

예제 6-7과 6-10을 비교해보자. <br>
<br>
6-10에서 instruction의 수가 훨씬 줄었는데, 이는 numpy의 벡터화 연산 덕분이다. 곱셈을 4번 하려고 명령을 4번 수행하는 대신 벡터화 연산으로 한 번에 수행하는 느낌. 또 다른 이유는 numpy는 모든 데이터를 수 형태로 저장한다고 가정하고 산술 연산에 최적화되어 있기 때문이다. <br>
<br>
numpy 버전에서 명령의 개수를 줄여 비효율적인 부분인 캐시 미스가 크게 개선되었다. (일반적으로는 명령어 개수가 적다고 해서 속도가 항상 빨라지는 것은 아니다.)

**예제 6-11** 벡터 연산 기능을 제외한 numpy 2차원 확산 방정식의 성능(격자 크기: 640x640, 500회 반복)

<span style="font-family: monospace">
~# perf stat -e cycles,instructions,cache-references,cache-misses,branches,branch-misses,task-clock,faults,minor-faults,cs,migrations python3 diffusion_numpy.py
</span>

In [ ]:
 Performance counter stats for 'python diffusion_numpy.py':

    50,086,999,350      cycles                    #    2.888 GHz                      
    53,611,608,977      instructions              #    1.07  insn per cycle           
     1,131,742,674      cache-references          #   65.266 M/sec                    
       322,483,897      cache-misses              #   28.494 % of all cache refs      
     4,001,923,035      branches                  #  230.785 M/sec                    
         6,211,101      branch-misses             #    0.16% of all branches          
      17340.464580      task-clock (msec)         #    1.000 CPUs utilized          
           403,193      faults                    #    0.023 M/sec                  
           403,193      minor-faults              #    0.023 M/sec                  
                74      cs                        #    0.004 K/sec                  
                 6      migrations                #    0.000 K/sec                  

      17.339656586 seconds time elapsed

결과를 보면, numpy를 사용해서 얻은 63.3배의 속도 개선은 벡터 연산 때문이라기보다 메모리 단편화의 감소와 메모리 지역성 덕분이라고 볼 수 있다. 실제로 이전 실험과 비교해보면 63.3배의 속도 개선 중에서 벡터 연산의 비중은 13%밖에 되지 않는다.

**예제 6-12** 메모리 할당을 줄이기 위한 제자리 연산

In [2]:
import numpy as np
array1 = np.random.random((10,10))
array2 = np.random.random((10,10))

In [3]:
id(array1)

2323650968528

In [4]:
array1 += array2
id(array1)

2323650968528

In [6]:
array1 = array1 + array2
id(array1)

2323655893136

**예제 6-13** 제자리 연산과 제자리 연산이 아닌 연산의 실행 시점 성능 차이

In [7]:
import numpy as np

In [8]:
%%timeit array1, array2 = np.random.random((2, 100, 100))
... array1 = array1 + array2

4.07 µs ± 82.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
%%timeit array1, array2 = np.random.random((2, 100, 100))
... array1 += array2

3.82 µs ± 50.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
%%timeit array1, array2 = np.random.random((2, 5, 5))
... array1 = array1 + array2

369 ns ± 3.35 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
%%timeit array1, array2 = np.random.random((2, 5, 5))
... array1 += array2

1.06 µs ± 2.76 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


- np.random.random((2, 100, 100)) 배열은 CPU 캐시에 들어가기에는 너무 크므로 할당과 캐시 미스가 적은 제자리 연산이 더 빠르다.
- np.random.random((2, 5, 5)) 배열은 캐시에 들어맞으므로 제자리 연산이 아닌 연산이 빠르다.
- %timeit 대신 %%timeit을 사용했음에 주목하자. %timeit은 실험 준비에 필요한 코드를 지정할 수 있게 해준다.

**예제 6-14** numpy 연산을 제자리 연산으로 변경

In [14]:
# 예제 6-9의 코드를 제자리 연산을 사용하도록 바꾸는 작업은 어렵지 않지만
# 코드의 가독성이 약간 떨어지는 문제가 있다.

def laplacian(grid, out):
    np.copyto(out, grid)
    out *= -4
    out += np.roll(grid, +1, 0)
    out += np.roll(grid, -1, 0)
    out += np.roll(grid, +1, 0)
    out += np.roll(grid, -1, 0)

    
def evolve(grid, dt, out, D=1):
    laplacian(grid, out)
    out *= D * dt
    out += grid


def run_experiment(num_iterations):
    scratch = zeros(grid_shape)
    grid = zeros(grid_shape)

    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)
    grid[block_low:block_high, block_low:block_high] = 0.005

    start = time.time()
    for i in range(num_iterations):
        evolve(grid, 0.1, scratch)     # evolve의 결과는 scratch 벡터에 저장되므로, grid와 scratch를 서로 바꿔치기 하여
        grid, scratch = scratch, grid  # 다음 반복에서 grid에 갱신된 정보가 저장되도록 한다. 이 바꿔치기는 데이터를 서로
    return time.time() - start        # 바꾸는게 아니라 데이터에 대한 참조만 바꾸므로 부담이 없다.

**예제 6-15** numpy와 제자리 메모리 연산을 사용했을 때의 성능 측정 결과 (격자 크기: 640x640, 500회 반복)

<span style="font-family: monospace">
~# perf stat -e cycles,instructions,cache-references,cache-misses,branches,branch-misses,task-clock,faults,minor-faults,cs,migrations python3 diffusion_numpy_memory.py
</span>

In [ ]:
 Performance counter stats for 'python diffusion_numpy_memory.py':

     6,880,906,446      cycles                    #    2.886 GHz                      
     5,848,134,537      instructions              #    0.85  insn per cycle           
     1,077,550,720      cache-references          #  452.000 M/sec                    
       217,974,413      cache-misses              #   20.229 % of all cache refs      
     1,028,769,315      branches                  #  431.538 M/sec                    
         7,492,245      branch-misses             #    0.73% of all branches          
       2383.962679      task-clock (msec)         #    1.373 CPUs utilized          
            13,521      faults                    #    0.006 M/sec                  
            13,521      minor-faults              #    0.006 M/sec                  
               100      cs                        #    0.042 K/sec                  
                 8      migrations                #    0.003 K/sec                  

       1.736322099 seconds time elapsed

**예제 6-16** laplacian 함수가 시간을 과하게 소비함을 알려주는 라인 프로파일링 결과

<span style="font-family: monospace">
~# kernprof -l -v diffusion_numpy_memory.py
</span>

In [ ]:
Wrote profile results to diffusion_numpy_memory.py.lprof
Timer unit: 1e-06 s

Total time: 2.02943 s
File: diffusion_numpy_memory.py
Function: evolve at line 24

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    24                                           @profile
    25                                           def evolve(grid, dt, out, D=1):
    26       500    1719414.0   3438.8     84.7      laplacian(grid, out)
    27       500     120996.0    242.0      6.0      out *= D * dt
    28       500     189021.0    378.0      9.3      out += grid

laplacian 함수가 느린 데는 여러 가지 이유가 있지만, 주요 원인은 두 가지이다. 첫째, np.roll을 호출하면서 새로운 벡터를 할당한다. 또한 np.roll은 매우 범용적인 함수로, 특수한 경우를 처리하기 위한 코드를 많이 포함한다. 우리는 단순히 축을 따라 데이터를 시프트하면 되므로 이 함수를 다시 작성해서 불필요한 코드를 제거할 수 있다. 

In [ ]:
 import time

from numpy import add, copyto, multiply, zeros

try:
    profile
except NameError:
    profile = lambda x: x

grid_shape = (640, 640)


def roll_add(rollee, shift, axis, out):
    """
    주어진 행렬 rollee에 대해서 다음 계산을 수행하여 결과를 out에 저장한다.
    
        >>> out += np.roll(rollee, shift, axis=axis)
    
    이 함수는 다음 가정하에 동작한다.
        * rollee는 2차원 배열이다.
        * shift는 +1 아니면 -1이다.
        * axis는 0 아니면 1이다. (rollee가 2차원 배열이므로 이를 유츄할 수 있다.)
        
    위와 같은 가정하에, numpy에서 범용적인 목적으로 구현한 roll 함수에서 불필요한
    부분을 회피하고 계산 과정을 함수 내부로 옮겨서 속도를 개선했다.
    """
    if shift == 1 and axis == 0:
        out[1:, :] += rollee[:-1, :]
        out[0, :] += rollee[-1, :]
    elif shift == -1 and axis == 0:
        out[:-1, :] += rollee[1:, :]
        out[-1, :] += rollee[0, :]
    elif shift == 1 and axis == 1:
        out[:, 1:] += rollee[:, :-1]
        out[:, 0] += rollee[:, -1]
    elif shift == -1 and axis == 1:
        out[:, :-1] += rollee[:, 1:]
        out[:, -1] += rollee[:, 0]


def laplacian(grid, out):
    copyto(out, grid)
    multiply(out, -4.0, out)
    roll_add(grid, +1, 0, out)
    roll_add(grid, -1, 0, out)
    roll_add(grid, +1, 1, out)
    roll_add(grid, -1, 1, out)


@profile
def evolve(grid, dt, out, D=1):
    laplacian(grid, out)
    multiply(out, D * dt, out)
    add(out, grid, out)


def run_experiment(num_iterations):
    scratch = zeros(grid_shape)
    grid = zeros(grid_shape)

    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)
    grid[block_low:block_high, block_low:block_high] = 0.005

    start = time.time()
    for i in range(num_iterations):
        evolve(grid, 0.1, scratch)
        grid, scratch = scratch, grid
    return time.time() - start


if __name__ == "__main__":
    run_experiment(500)

**예제 6-18** numpy를 이용한 제자리 메모리 연산과 사용자 정의 laplacian 함수를 사용했을 때 성능 측정 결과 (격자 크기: 640x640, 500회 반복)

<span style="font-family: monospace">
~# perf stat -e cycles,instructions,cache-references,cache-misses,branches,branch-misses,task-clock,faults,minor-faults,cs,migrations python3 diffusion_numpy_memory2.py
</span>

In [ ]:
 Performance counter stats for 'python diffusion_numpy_memory2.py':

     5,971,464,515      cycles                    #    2.888 GHz                      
     5,893,131,049      instructions              #    0.99  insn per cycle           
     1,001,582,133      cache-references          #  484.398 M/sec                    
        30,840,612      cache-misses              #    3.079 % of all cache refs      
     1,038,649,694      branches                  #  502.325 M/sec                    
         7,562,009      branch-misses             #    0.73% of all branches          
       2067.685884      task-clock (msec)         #    1.456 CPUs utilized          
            11,981      faults                    #    0.006 M/sec                  
            11,981      minor-faults              #    0.006 M/sec                  
                95      cs                        #    0.046 K/sec                  
                 3      migrations                #    0.001 K/sec                  

       1.419869071 seconds time elapsed

예제 6-18과 6-14를 비교해보자. <br>
<br>
대부분 항목이 비슷함에도 22% 더 빨라진 것을 확인할 수 있다. 가장 큰 차이는 cahce-misses로, 7배 줄었다. 이런 변화는 CPU의 명령어 처리율(throughput)에도 영향을 주어서, 사이클 당 명령어 수가 0.85에서 0.99로 늘었다. 비슷하게 faults는 12.85% 줄었다. <br>
<br>
이런 결과는 행렬을 시프트할 때 numpy에 전적으로 의존하는 대신 제자리에서 데이터를 갱신하는 함수를 작성했기 때문이다. numpy는 배열을 바로 내보내지 않고, 경계 검사나 오류 처리 등 처리 작업에 필요한 다른 검사를 실행한 후 결과를 추가한다. 우리가 만든 함수는 한 번에 모든 작업을 처리하며 컴퓨터가 매번 캐시를 채워 넣을 필요가 없다.

## 6.5 numexpr: 제자리 연산을 더 빠르고 간편하게 쓰기

numpy의 벡터 연산 최적화는 한 번에 하나의 연산에만 적용할 수 있다는 단점이 있다. 즉, numpy 벡터로 A * B + C를 계산하면, 먼저 A * B 계산 결과를 임시 벡터에 저장한 다음에 C를 더한다. <br>
<br>
numexpr 모듈은 계산에 사용한 전체 벡터를 취합해서 캐시 미스와 임시 공간 사용을 최소화하는 아주 효율적인 코드로 컴파일한다. 멀티 코어를 활용할 수도 있고, CPU가 지원하는 특수한 명령어를 사용할 수도 있다. 여러 코어에 연산을 병렬화해주는 OpenMP도 지원한다. <br>
<br>
코드에서 numexpr을 사용하도록 바꾸려면, 문자열 표현으로 지역 변수를 참조하기만 하면 된다.

**예제 6-19** numexpr을 사용해서 대규모 행렬 연산 최적화

In [ ]:
from numexpr import evaluate

def evolve(grid, dt, out, D=1):
    laplacian(grid, out)
    ne.evaluate("out*D*dt+grid", out=out)

**예제 6-20** numpy의 제자리 메모리 연산, 사용자 정의 laplacian 함수, numexpr을 사용했을 때의 성능 측정 결과 (격자 크기: 640x640, 500회 반복)

<span style="font-family: monospace">
~# perf stat -e cycles,instructions,cache-references,cache-misses,branches,branch-misses,task-clock,faults,minor-faults,cs,migrations python3 diffusion_numpy_memory2_numexpr.py
</span>

In [ ]:
 Performance counter stats for 'python diffusion_numpy_memory2_numexpr.py':

     8,856,947,179      cycles                    #    2.872 GHz                      
     9,354,357,453      instructions              #    1.06  insn per cycle           
     1,077,518,384      cache-references          #  349.423 M/sec                    
        59,407,830      cache-misses              #    5.513 % of all cache refs      
     1,018,525,317      branches                  #  330.292 M/sec                    
        11,941,430      branch-misses             #    1.17% of all branches          
       3083.709890      task-clock (msec)         #    1.991 CPUs utilized          
            15,820      faults                    #    0.005 M/sec                  
            15,820      minor-faults              #    0.005 M/sec                  
             8,671      cs                        #    0.003 M/sec                  
             2,096      migrations                #    0.680 K/sec                  

       1.548924090 seconds time elapsed

예제 6-18과 6-20을 비교해보면 numexpr을 사용하지 않았을 때가 오히려 더 빠르다. 계산에 필요한 모든 데이터가 캐시에 저장될 만큼 격자가 작다면 이런 일이 생긴다.

## 6.6 경고: '최적화' 검증(사이파이)

이 장에서 중요한 내용은 최적화 작업을 하면서 프로파일링으로 코드가 어떻게 동작하는지 감을 잡고, 느린 부분을 개선할 수 있는 **해법**을 적용한 다음, 다시 프로파일해서 느린 부분을 제대로 개선했는지 확인하는 것이다.

**예제 6-21** scipy의 laplace 필터 사용

In [ ]:
from scipy.ndimage.filters import laplace

def laplacian(grid, out):
    laplace(grid, out, mode="wrap")

위와 같이 코드를 수정하면, scipy에는 이미지 처리용 하위 모듈이 있으므로 분명 도움이 될 것이라고 기대할 수도 있다는데 (나는 기대 안했음. 애초에 scipy가 뭔지도 몰랐음.) 실제로는 그렇지 않다고 함.

**예제 6-22** scipy의 laplace 함수를 사용한 확산 방정식의 성능 측정 결과 (격자 크기: 640x640, 500회 반복)

<span style="font-family: monospace">
~# perf stat -e cycles,instructions,cache-references,cache-misses,branches,branch-misses,task-clock,faults,minor-faults,cs,migrations python3 diffusion_scipy.py
</span>

In [ ]:
 Performance counter stats for 'python diffusion_scipy.py':

    10,051,801,725      cycles                    #    2.886 GHz                      
    16,536,981,020      instructions              #    1.65  insn per cycle           
     1,554,557,564      cache-references          #  446.405 M/sec                    
       126,627,735      cache-misses              #    8.146 % of all cache refs      
     2,673,416,633      branches                  #  767.696 M/sec                    
         9,626,762      branch-misses             #    0.36% of all branches          
       3482.391211      task-clock (msec)         #    1.228 CPUs utilized          
            14,013      faults                    #    0.004 M/sec                  
            14,013      minor-faults              #    0.004 M/sec                  
                95      cs                        #    0.027 K/sec                  
                 5      migrations                #    0.001 K/sec                  

       2.835263796 seconds time elapsed

 예제 6-18이나 6-20과 비교해보자. instruction 지표가 가장 많이 차이나는데, scipy 코드는 경계 조건이 다양한 모든 종류의 입력을 처리할 수 있도록 작성되었으므로 이런 결과가 생겼다. 같은 이유로 branch 항목이 더 높은 것을 확인할 수 있다.

## 6.7 행렬 최적화에서 얻은 교훈

**표 6-1** 격자 크기 별 evolve 함수(500회 반복) 실행 시간 비교 (단위: 초)

| 방법                                 | 256 x 256 | 512 x 512 | 1,024 x 1,024 | 2,048 x 2,048 | 4,096 x 4,096 |
|:-------------------------------------|:----------|:----------|:--------------|:--------------|:--------------|
| 파이썬                               | 2.40      | 10.43     | 41.75         | 168.82        | 1679.16       |
| 파이썬 + 메모리                      | 2.26      | 9.76      | 38.85         | 157.25        | 632.76        |
| numpy                                | 0.01      | 0.09      | 0.69          | 3.77          | 14.83         |
| numpy + 메모리                       | 0.01      | 0.07      | 0.60          | 3.80          | 14.97         |
| numpy + 메모리 + laplacian           | 0.01      | 0.05      | 0.48          | 1.86          | 7.50          |
| numpy + 메모리 + laplacian + numexpr | 0.02      | 0.06      | 0.41          | 1.60          | 6.45          |
| numpy + 메모리 + scipy               | 0.05      | 0.25      | 1.15          | 6.83          | 91.43         |

초기화 과정에 필요한 관리 요소를 모두 고려해야 한다. 여기에는 메모리 할당, 설정 파일을 읽는 과정, 프로그램이 실행되는 동안 필요한 값을 미리 계산하는 과정이 모두 포함된다. <br>
<br>
그리고 이것은 다음의 두 가지 이유에서 중요하다. 첫째, 이런 초기화 작업을 한 번에 완료해서 전체 실행 횟수를 줄이고 나중에 불필요한 자원이 낭비되지 않도록 한다. 둘째, 프로그램의 흐름을 방해하지 않아야 한다. 캐시에 관련 데이터만 채워지게 하고 파이프라인을 효율적으로 유지할 수 있다.

## 6.8 팬더스

팬더스는 과학기술 파이썬 생태계에서 사실상 표준인 데이터 조작 도구로 표 형태의 데이터 처리에 사용한다.

### 6.8.1 팬더스 내부 모델

DataFrame에 대한 연산은 한 열에 속한 모든 셀(또는 axis=1 인자 사용 시 한 행에 속한 모든 셀)에 적용된다. <br>
<br>
팬더스는 원래 numpy 데이터 타입을 사용했지만, 발전하는 과정에서 3가 논리의 '없는 데이터(NaN)' 동작을 인식하는 팬더스 자체 데이터타입이 늘어났다. 그래서 numpy의 int64(NaN을 인식하지 못한다)와 팬더스 Int64(내부적으로 정수와 NaN을 표현하는 Bit 마스크로 이뤄진 2열 데이터를 사용한다)를 구분해야한다. 참고로 numpy의 float64는 NaN을 인식한다. 

### 6.8.2 여러 데이터 행에 함수 적용하기

**일반 최소 제곱(OLS)**

데이터를 직선에 맞추는 방법. 주어진 데이터 집합에 대해 m * x + c 공식의 기울기와 절편을 계산한다.

**예제 6-23** 예제 데이터

10만 행으로 이뤄진 데이터를 생성한다. 각 행은 인공적인 고객을 나타내며, 행마다 열이 14개 있다. 각 열은 '매일 휴대폰을 사용한 시간'을 나타내는 연속적인 변수이다. <br>
<br>
푸아송 분포(lambda==60, 단위는 분)를 그리고, 60으로 나눠서 연속적인 변수를 얻어 시간을 시뮬레이션한다.

**어떤 OLS 구현을 사용해야 할까?**

**예제 6-24** OLS를 넘파이와 사이킷런으로 풀기

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

def ols_sklearn(row):
    """사이킷런의 LinearRegression로 OLS 풀기"""
    est = LinearRegression() 
    X = np.arange(row.shape[0]).reshape(-1, 1) # shape (14, 1)
    # LinearRegression 내에서 절편이 만들어진다
    est.fit(X, row.values) 
    m = est.coef_[0] # c가 est.intercept_ 이다
    return m

def ols_lstsq(row):
    """numpy.linalg.lstsq로 OLS 풀기"""
    # build X values for [0, 13]
    X = np.arange(row.shape[0]) # shape (14,)
    ones = np.ones(row.shape[0]) # 절편을 만들려 사용한 상수
    A = np.vstack((X, ones)).T # shape(14, 2)
    # lstsq는 계수를 반환하며, 첫번째 계수는 절편이고 
    # 그 이후는 리지듀얼과 다르요소들이다
    m, c = np.linalg.lstsq(A, row.values, rcond=-1)[0] 
    return m

def ols_lstsq_raw(row):
    """row가 numpy 배열 (Series가 아님)인 `ols_lstsq` 변종"""
    X = np.arange(row.shape[0])
    ones = np.ones(row.shape[0])
    A = np.vstack((X, ones)).T
    m, c = np.linalg.lstsq(A, row, rcond=-1)[0] 
    return m

**예제 6-25** 사이킷런의 LinearRegression.fit 호출 상세 분석

In [ ]:
...
lp = LineProfiler(est.fit)
print("Run on a single row")
lp.run("est.fit(X, row.values)")
lp.print_stats()

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   491                                               def fit(self, X, y, sample_weight=None):
...
   516         1         20.0     20.0      0.2          accept_sparse = False if self.positive else ['csr', 'csc', 'coo']
   517                                           
   518         2       3594.0   1797.0     40.5          X, y = self._validate_data(X, y, accept_sparse=accept_sparse,
   519         1         12.0     12.0      0.1                                     y_numeric=True, multi_output=True)
   520                                           
   521         1         20.0     20.0      0.2          if sample_weight is not None:
   522                                                       sample_weight = _check_sample_weight(sample_weight, X,
   523                                                                                            dtype=X.dtype)
   524                                           
   525         2       2486.0   1243.0     28.0          X, y, X_offset, y_offset, X_scale = self._preprocess_data(
   526         1         14.0     14.0      0.2              X, y, fit_intercept=self.fit_intercept, normalize=self.normalize,
   527         1         14.0     14.0      0.2              copy=self.copy_X, sample_weight=sample_weight,
   528         1         11.0     11.0      0.1              return_mean=True)
...
   567                                                   else:
   568         1         22.0     22.0      0.2              self.coef_, self._residues, self.rank_, self.singular_ = \
   569         1       2236.0   2236.0     25.2                  linalg.lstsq(X, y)
   570         1         20.0     20.0      0.2              self.coef_ = self.coef_.T
   571                                           
   572         1         14.0     14.0      0.2          if y.ndim == 1:
   573         1        112.0    112.0      1.3              self.coef_ = np.ravel(self.coef_)
   574         1        201.0    201.0      2.3          self._set_intercept(X_offset, y_offset, X_scale)
   575         1         12.0     12.0      0.1          return self


**행 데이터에 lstsq 적용하기**

**예제 6-26** 가장 나쁜 구현: iloc를 사용해 한번에 한 행씩 가져와서 계산하기

In [ ]:
ms = []
for row_idx in range(df.shape[0]):
    row = df.iloc[row_idx]
    m = ols_lstsq(row)
    ms.append(m)
results = pd.Series(ms)

iloc은 새 row_idx에서 행을 가져오려고 아주 많은 작업을 수행한다. 그 후 가져온 행을 새 Series 객체로 변환해 반환하고, 이를 row에 대입한다.

**예제 6-27** iterrows를 사용해 더 파이썬답고 효율적으로 행 연산하기

In [ ]:
ms = []
for row_idx, row in df.iterrows():
    m = ols_lstsq(row)
    ms.append(m)
results = pd.Series(ms)

5장에서 이터레이션하는 감성이다. 여전히 반복마다 새로운 Series 객체를 만들어서 row에 저장한다.

**예제 6-28** apply를 사용해 전형적인 팬더스 함수 적용하기

In [ ]:
ms = df.apply(ols_lstsq, axis=1)
results = pd.Series(ms)

apply는 새로운 파이썬 중간 참조를 만들지 않고 ols_lstsq를 데이터 행에 직접 넘긴다. 여기서도 내부에서는 행마다 새 Series가 만들어진다.

**예제 6-29** raw=True를 사용해 중간 Series 생성 방지하기

In [ ]:
ms = df.apply(ols_lstsq_raw, axis=1, raw=True)
results = pd.Series(ms)

apply를 사용할 때 raw=True를 인자로 사용하면 중간 Series 객체가 만들어지지 않는다. Series 객체가 없으므로 ols_lstsq_raw를 사용해야 한다.

**표 6-3** lstsq와 다양한 팬더스 행 단위 접근 방법을 사용한 경우의 실행 시간 비교

| 방법           | 시간(초) |
|:---------------|:---------|
| iloc           | 18.6     |
| iterrows       | 12.4     |
| apply          | 6.8      |
| apply raw=True | 5.3      |

### 6.8.3 부분 결과를 이어 붙이지 않고 DataFrame과 Series 만들기

**예제 6-30** 각 결과를 이어 붙이면 부가비용이 많이 든다. (이렇게 하지 말라!)

In [ ]:
results = None
for row_idx in range(df.shape[0]):
    row = df.iloc[row_idx]
    m = ols_lstsq(row)
    if results is None:
        results = pd.Series([m])
    else:
        #results = results.append(pd.Series([m]))  # equivalent to concat
        results = pd.concat((results, pd.Series([m])))

### 6.8.4 어떤 일을 하는 또 다른(게다가 더 빠르) 방법

**예제 6-31** 문자열 처리에서 str Series 연산과 apply의 비교

In [ ]:
df['0_as_str'] = df[0].apply(lambda v: str(v))

def find_9(s): 
    """'9'를 못 찾으면 -1, 그렇지 않으면 발견한 위치(위치 >= 0)를 돌려준다"""
    return s.split('.')[1].find('9')

#%timeit df['0_as_str'].str.split('.', expand=True)[1].str.find('9')
#183 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
#%timeit df['0_as_str'].apply(find_9)
#51 ms ± 987 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)